In [5]:
import geopandas as gpd
import pandas as pd

# Cargar las rutas desde el archivo SHP
gdf_rutas = gpd.read_file(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\archivo_atributos_unificados.shp")

# Cargar las mallas de la Península y de Canarias
gdf_malla_peninsula = gpd.read_file(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos IEET\Mallas Península\Malla10km_p.shp")
gdf_malla_canarias = gpd.read_file(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos IEET\Mallas Canarias\Malla10km_c.shp")

# Cargar el archivo CSV de las rutas
df_rutas_csv = pd.read_csv(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\archivo_rutas.csv")


In [6]:
# Asegurarse de que ambos GeoDataFrames usan el mismo CRS
gdf_rutas = gdf_rutas.to_crs("EPSG:4326")
gdf_malla_peninsula = gdf_malla_peninsula.to_crs("EPSG:4326")
gdf_malla_canarias = gdf_malla_canarias.to_crs("EPSG:4326")

In [22]:
gdf_rutas_canarias_test = gpd.sjoin(gdf_rutas, gdf_malla_canarias, how="left", predicate="intersects")

print(gdf_rutas_canarias_test['CUADRICULA'].head())

0    28RCS53
0    28RCS54
0    28RCS64
1    28RCS43
1    28RCS53
Name: CUADRICULA, dtype: object


In [ ]:
# Unir las rutas con la malla de la Península
gdf_rutas_peninsula = gpd.sjoin(gdf_rutas, gdf_malla_peninsula, how="left", predicate="intersects")

# Unir las rutas con la malla de Canarias
gdf_rutas_canarias = gpd.sjoin(gdf_rutas, gdf_malla_canarias, how="left", predicate="intersects")

# Unir los resultados de las rutas en Península y Canarias
gdf_rutas_combinadas = pd.concat([gdf_rutas_peninsula, gdf_rutas_canarias], ignore_index=True)

# Eliminar filas duplicadas (en caso de que una ruta esté en ambas mallas)
gdf_rutas_combinadas = gdf_rutas_combinadas.drop_duplicates(subset='ID_Ruta')  # Usa el identificador único de cada ruta

# Ver las primeras filas para comprobar el resultado
print(gdf_rutas_combinadas[['ID_Ruta', 'CUADRICULA']].head())


0    28RCS53
0    28RCS54
0    28RCS64
1    28RCS43
1    28RCS53
Name: CUADRICULA, dtype: object
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: CUADRICULA, dtype: object


In [15]:
# Unir el DataFrame original de rutas con el GeoDataFrame combinado
df_rutas_con_malla = pd.merge(df_rutas_csv, gdf_rutas_combinadas[['ID_Ruta', 'CUADRICULA']], 
                               on='ID_Ruta', how='left')

# Reordenar las columnas colocando 'CUADRICULA' al inicio
columnas = ['CUADRICULA'] + [col for col in df_rutas_con_malla.columns if col != 'CUADRICULA']
df_rutas_con_malla = df_rutas_con_malla[columnas]

# Verificar el resultado
print(df_rutas_con_malla.head())

  CUADRICULA  ID_Ruta                       Nombre_Ruta  \
0        NaN        0  Camino Natural de Anaga - Chasna   
1        NaN        1  Camino Natural de Anaga - Chasna   
2        NaN        2  Camino Natural de Anaga - Chasna   
3        NaN        3  Camino Natural de Anaga - Chasna   
4        NaN        4  Camino Natural de Anaga - Chasna   

                                           NOM_ETAPA  \
0                 Etapa 1: La Esperanza - La Caldera   
1                  Etapa 2: La Caldera - El Portillo   
2  Etapa 3: El Portillo - Parador de Las Cañadas ...   
3  Etapa 4: Parador de Las Cañadas del Teide - Vi...   
4                          Etapa 5: Vilaflor - Arona   

                                     Nombre_completo       Longitud  \
0  Camino Natural de Anaga - Chasna Etapa 1: La E...  30,9240811977   
1  Camino Natural de Anaga - Chasna Etapa 2: La C...  14,1712623036   
2  Camino Natural de Anaga - Chasna Etapa 3: El P...   16,686617195   
3  Camino Natural de Ana

In [16]:
# Definir la ruta de salida
ruta_salida = r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas/archivo_rutas_con_cuadricula.csv"

# Guardar el archivo CSV con la nueva columna
df_rutas_con_malla.to_csv(ruta_salida, index=False, encoding="utf-8-sig", decimal=",")
print(f"Archivo de rutas con cuadricula guardado en: {ruta_salida}")

Archivo de rutas con cuadricula guardado en: C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas/archivo_rutas_con_cuadricula.csv
